## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo Rides</strong>

<strong>3. Pooled Rides</strong>

## Load ExMAS and MaaSSim


In [19]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
import logging
import matplotlib.pyplot as plt

import pandas as pd
import ExMAS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Delft, Netherlands

In [20]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 3600 # 1 hour of simulation
params.simTime = 1 # 6 minutes hour of simulation
params.nP = 50 # reuqests (and passengers)
params.nV = 20 # vehicles


## Parameters for ExMAS

In [21]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.0005
params.shareability.comm_rate = 0.2

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True



# params.kpi = 1 (Profit Maximazation)
# params.kpi = 2 (Solo Rides)
# params.kpi = 3 (Pooled Rides)


In [30]:
params.kpi = 1

### Profit Mazimization - Begin 

In [31]:
#inData = ExMAS.main(inData, params.shareability) # create shareability graph (ExMAS) 
inData = ExMAS.main(inData, params.shareability, plot = False)

13-07-23 09:28:51-INFO-Initializing pairwise trip shareability between 50 and 50 trips.
13-07-23 09:28:51-INFO-creating combinations
13-07-23 09:28:51-INFO-2450	 nR*(nR-1)
13-07-23 09:28:51-INFO-Reduction of feasible pairs by 99.51%
13-07-23 09:28:51-INFO-Degree 2 	Completed
13-07-23 09:28:51-INFO-trips to extend at degree 2 : 25
13-07-23 09:28:51-INFO-At degree 2 feasible extensions found out of 0 searched
13-07-23 09:28:51-INFO-Degree 3 	Completed
13-07-23 09:28:51-INFO-No more trips to exted at degree 3


In [32]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

13-07-23 09:28:52-INFO-Matching 50 trips to 75 rides in order to min u_veh
13-07-23 09:28:52-INFO-Problem solution: Optimal. 
Total costs for single trips:         12,093 
reduced by matching to:               11,304


In [33]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],8.411000,454,1,[8.411],"[0, 454]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1530303614, 1566353769]","[None, 0, 0]",node time req_id od 0 ...,454,4.54,6.822000,1.364400,5.4576
1,[1],1.395000,75,1,[1.395],"[69, 75]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 839335227, 44790565]","[None, 1, 1]",node time req_id od 0 N...,75,0.75,1.132500,0.226500,0.9060
2,[2],3.047500,164,1,[3.0475000000000003],"[130, 164]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 2408303441, 44822077]","[None, 2, 2]",node time req_id od 0 ...,164,1.64,2.473500,0.494700,1.9788
3,[3],6.077000,328,1,[6.077],"[289, 328]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44727971, 1571088109]","[None, 3, 3]",node time req_id od 0 ...,328,3.28,4.929000,0.985800,3.9432
4,[4],1.797500,97,1,[1.7974999999999999],"[335, 97]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44807530, 643586468]","[None, 4, 4]",node time req_id od 0 N...,97,0.97,1.458000,0.291600,1.1664
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,"[34, 42]",11.017450,626,21,"[9.168725, 1.848725]","[2353.5, 312, 116, 198]","[34, 42]","[42, 34]",2,70,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 736792569, 311132084, 5119770538, 15526...","[None, 34, 42, 42, 34]",node time req_id od 0 ...,626,6.26,7.609500,1.521900,6.0876
71,"[34, 44]",10.966500,652,21,"[9.4349, 1.5316]","[2314.0, 512, 86, 54]","[34, 44]","[44, 34]",2,71,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 736792569, 1258471011, 565046992, 15526...","[None, 34, 44, 44, 34]",node time req_id od 0 ...,652,6.52,7.281000,1.456200,5.8248
72,"[36, 38]",8.463825,436,21,"[6.203175, 2.26065]","[2436.0, 213, 133, 90]","[36, 38]","[38, 36]",2,72,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1570983285, 44754398, 4655063484, 14139...","[None, 36, 38, 38, 36]",node time req_id od 0 ...,436,4.36,5.556375,1.111275,4.4451
73,"[36, 44]",7.616475,481,21,"[6.248675, 1.3678000000000001]","[2471.0, 319, 86, 76]","[36, 44]","[44, 36]",2,73,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1570983285, 1258471011, 565046992, 1413...","[None, 36, 44, 44, 36]",node time req_id od 0 ...,481,4.81,5.036625,1.007325,4.0293


In [34]:
sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate

13-07-23 09:28:53-INFO-Matching 50 trips to 75 rides in order to min u_veh
13-07-23 09:28:53-INFO-Problem solution: Optimal. 
Total costs for single trips:         12,093 
reduced by matching to:               11,304
0.0005
13-07-23 09:28:53-CRITICAL-this is request 0 with [0, 50, 63] available rides.
13-07-23 09:28:53-CRITICAL-ride 0 available [1]
13-07-23 09:28:53-CRITICAL-ride 50 available [1, 0]
13-07-23 09:28:53-CRITICAL-ride 63 available [1, 0]
13-07-23 09:28:53-CRITICAL-this is reuqest 0 with [0, 50, 63] still available rides.
13-07-23 09:28:53-CRITICAL-vehicle 1 has 3 choices
0.0005
13-07-23 09:28:53-CRITICAL-this is request 1 with [1] available rides.
13-07-23 09:28:53-CRITICAL-ride 1 available [1]
13-07-23 09:28:53-CRITICAL-this is reuqest 1 with [1] still available rides.
13-07-23 09:28:53-CRITICAL-vehicle 3 has 1 choices
0.0005
13-07-23 09:28:53-CRITICAL-this is request 2 with [2, 63] available rides.
13-07-23 09:28:53-CRITICAL-ride 2 available [1]
13-07-23 09:28:53-CRITICA

In [35]:
sim.res[0].veh_exp

,nRIDES,nREJECTED,OUT,STARTS_DAY,OPENS_APP,RECEIVES_REQUEST,ACCEPTS_REQUEST,REJECTS_REQUEST,IS_ACCEPTED_BY_TRAVELLER,IS_REJECTED_BY_TRAVELLER,...,STARTS_REPOSITIONING,REPOSITIONED,DECIDES_NOT_TO_DRIVE,ENDS_SHIFT,TRAVEL,WAIT,CRUISE,OPERATIONS,IDLE,PAX_KM
1,4.0,0.0,False,0,0,1206.0,45.0,0,188.0,0,...,0,0,0,7794.0,1158.0,0,288.0,353.0,5995.0,12.848
2,0.0,0.0,False,0,0,0.0,0.0,0,0.0,0,...,0,0,0,10799.0,0.0,0,0.0,0.0,10799.0,0.000
3,2.0,0.0,False,0,0,231.0,30.0,0,40.0,0,...,0,0,0,9949.0,385.0,0,104.0,130.0,9330.0,3.855
4,2.0,0.0,False,0,0,2567.0,30.0,0,40.0,0,...,0,0,0,7830.0,185.0,0,87.0,130.0,7428.0,1.862
5,0.0,0.0,False,0,0,0.0,0.0,0,0.0,0,...,0,0,0,10799.0,0.0,0,0.0,0.0,10799.0,0.000
6,4.0,0.0,False,0,0,1249.0,60.0,0,80.0,0,...,0,0,0,8480.0,479.0,0,331.0,260.0,7410.0,4.802
7,3.0,0.0,False,0,0,1347.0,45.0,0,45.0,0,...,0,0,0,8367.0,618.0,0,287.0,180.0,7282.0,6.201
8,1.0,0.0,False,0,0,396.0,15.0,0,20.0,0,...,0,0,0,9977.0,347.0,0,14.0,65.0,9551.0,3.472
9,4.0,0.0,False,0,0,2321.0,45.0,0,203.0,0,...,0,0,0,7050.0,597.0,0,463.0,368.0,5622.0,8.583
10,3.0,0.0,False,0,0,1895.0,30.0,0,81.0,0,...,0,0,0,8004.0,390.0,0,309.0,201.0,7104.0,5.182


In [45]:
sim.vehs[6].rdf

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,sim_schedule,ttrav,dist,fare,commission,driver_revenue,pickup_dist,trav_dist,operating_cost,profit
52,"[4, 7]",7.28825,486,20,"[1.755525, 5.532725]","[353.5, 55, 72, 359]","[4, 7]","[4, 7]",2,52,...,node time req_id od 0 ...,486,4.86,4.58100,0.91620,3.6648,789,793.86,0.396930,3.267870
55,"[15, 22]",9.71720,561,20,"[5.210675, 4.506525]","[1136.0, 274, 56, 231]","[15, 22]","[15, 22]",2,55,...,node time req_id od 0 ...,561,5.61,6.08175,1.21635,4.8654,2396,2401.61,1.200805,3.664595
16,[16],1.81600,98,1,[1.816],"[1052, 98]",[16],[16],1,16,...,node time req_id od 0 ...,98,0.98,1.47300,0.29460,1.1784,989,989.98,0.494990,0.683410
22,[22],4.58000,247,1,[4.58],"[1501, 247]",[22],[22],1,22,...,node time req_id od 0 ...,247,2.47,3.71550,0.74310,2.9724,1197,1199.47,0.599735,2.372665
31,[31],0.68600,37,1,[0.6859999999999999],"[2122, 37]",[31],[31],1,31,...,node time req_id od 0 ...,37,0.37,0.55650,0.11130,0.4452,1107,1107.37,0.553685,-0.108485


In [46]:
pd.DataFrame(sim.vehs[6].myrides)

,veh,pos,t,event,paxes
0,6,3.773543e+09,0,STARTS_DAY,[]
1,6,3.773543e+09,0,OPENS_APP,[]
2,6,3.773543e+09,335,RECEIVES_REQUEST,[]
3,6,3.773543e+09,350,ACCEPTS_REQUEST,[]
4,6,3.773543e+09,370,IS_ACCEPTED_BY_TRAVELLER,[None]
5,6,4.480753e+07,431,ARRIVES_AT_PICKUP,[]
6,6,4.480753e+07,431,MEETS_TRAVELLER_AT_PICKUP,[]
7,6,4.480753e+07,461,DEPARTS_FROM_PICKUP,[4]
8,6,6.435865e+08,558,ARRIVES_AT_DROPOFF,[4]
9,6,6.435865e+08,1045,RECEIVES_REQUEST,[]


In [404]:
responses = []
avg_kpi = []
idle_time = []

for i in range(2, 3):
    params.kpi = i
    sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/veh{}.csv'.format(i))
    sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/pax{}.csv'.format(i))
    #driver_data.loc['Cost'] = driver_data.loc['Revenue'].apply(lambda x: x*params.shareability.operating_cost)['Vehicles'] = sim.res[0].veh_exp.index
    #sim.res[0].veh_exp['ds'] = f"{i}"
    
    #responses.append(sim.res[0].veh_exp)
     
   # vehicles = sim.res[0].veh_exp.loc[sim.res[0].veh_exp["nRIDES"] > 0]
    #no_of_veh = len(vehicles)
    
    #avg_kpi.append(sim.res[0].all_kpi/no_of_veh)
    #idle_time.append(vehicles['IDLE'].sum()/no_of_veh)
    
    #_inData.sblts.rides['commission'] = _inData.sblts.rides.apply(lambda row: row.fare*sp.comm_rate, axis=1)
    

[autoreload of MaaSSim.performance failed: Traceback (most recent call last):
  File "C:\Users\usman\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\extensions\autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "C:\Users\usman\AppData\Local\Programs\Python\Python310\lib\site-packages\IPython\extensions\autoreload.py", line 455, in superreload
    module = reload(module)
  File "C:\Users\usman\AppData\Local\Programs\Python\Python310\lib\importlib\__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 1017, in get_code
  File "<frozen importlib._bootstrap_external>", line 947, in source_to_code
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "D:\Development\GitHub-ProjectV2.0\MaaSSim\MaaSSim\performa

0.0005
12-07-23 10:08:51-CRITICAL-this is request 0 with [0, 50, 51, 80, 81, 82, 105, 106] available rides.
12-07-23 10:08:51-CRITICAL-ride 0 available [1]
12-07-23 10:08:51-CRITICAL-ride 50 available [1, 0]
12-07-23 10:08:51-CRITICAL-ride 51 available [1, 0]
12-07-23 10:08:51-CRITICAL-ride 80 available [1, 0]
12-07-23 10:08:51-CRITICAL-ride 81 available [1, 0]
12-07-23 10:08:51-CRITICAL-ride 82 available [1, 0]
12-07-23 10:08:51-CRITICAL-ride 105 available [1, 0, 0]
12-07-23 10:08:51-CRITICAL-ride 106 available [1, 0, 0]
12-07-23 10:08:51-CRITICAL-this is reuqest 0 with [0, 50, 51, 80, 81, 82, 105, 106] still available rides.
12-07-23 10:08:51-CRITICAL-vehicle 1 has 8 choices
0.0005
12-07-23 10:08:51-CRITICAL-this is request 1 with [1, 52, 80, 83, 105] available rides.
12-07-23 10:08:51-CRITICAL-ride 1 available [1]
12-07-23 10:08:51-CRITICAL-ride 52 not available [1, 3]
12-07-23 10:08:51-CRITICAL-ride 80 not available [3, 1]
12-07-23 10:08:51-CRITICAL-ride 83 not available [3, 1]
12-